# Finding the next NBA City

### Introduction

The sports entertainment industry is a multi-billion dollar industry. Comprised of four major sports leagues (the NFL, the NBA, MLB and the NHL) each with roughly 30 teams, the impact of these leagues is felt around the country. Yet, in spite of the seeming national saturation of sports teams and leagues, there is continual talk of adding additional teams to new cities to further expand their respective sports. 

A league that has seen growing popularity as of late is the National Basketball Association (NBA). In light of this, talk has been swirling of adding an expansion team to a new market. To further fuel these rumors, the NBA has relatively fewer teams than its two Fall/Winter sports league rivals; 30 teams vs 32 for the  NFL and (soon to be) 32 for the NHL.

With this as a backdrop, the NBA is looking to expand its team base and, therefore, fan and revenue base via expansion. The challenge is finding a city that is suitable for expansion and will create a corresponding increase in revenue without diluting the current product. 

The goal of this study is to help the NBA narrow down its search for suitable expansion cities. This will be done by looking at both bulk city data (population, TV market size, income) and the interests of the locals via popular venues and locales within the respective cities. 

In [4]:
# make the imports
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import sys
#!{sys.executable} -m pip install lxml
#import lxml

### Data

In addition to the FourSqure locale data, there are 4 data sources we need:

1. City Population Data - https://en.m.wikipedia.org/wiki/List_of_United_States_cities_by_population
2. TV Market Data - https://en.wikipedia.org/wiki/List_of_United_States_television_markets
3. Metro Area income data - https://en.wikipedia.org/wiki/List_of_United_States_metropolitan_areas_by_per_capita_income
4. List of current NBA teams - https://www.basketball-reference.com/teams/ 

The first three are located in Wikipedia and will need to be scraped separately then merged for later use.

The last one was exported as a .csv file through the basketball-reference website and will be read-in to a pandas dataframe directly

#### Population Data

First, we'll pull in the population data from Wikipedia

In [5]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

OptionError: "No such keys(s): 'display.height'"

In [6]:
pop_wiki = requests.get("https://en.m.wikipedia.org/wiki/List_of_United_States_cities_by_population")
pop_soup = BeautifulSoup(pop_wiki.content,'html')
#pop_soup

/usr/lib/python3/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [7]:
# get the table reference
# By inspection of HTML, table we are interested in is stored in index 3
table = pop_soup.find_all('table')[3]
#table

We will define a function for extracting the table into a dataframe for the population data wiki page

In [8]:
def get_table_data(table):
    
    # Extract the column names
    columns = []
    for c in table.find_all('th'):
        columns.append(c.get_text().strip())
   
    # get the remaining elements based on the length of the column headers
    num_cols = len(columns)
    # create an empty dataframe with the columns discovered
    df = pd.DataFrame()
    
    i=0
    row=[]
    
    for td in table.find_all('td'):
        #print(td.get_text())
        if ('sq' in td.get_text()) and ('mi' in td.get_text()):
            i=i-1
        else:
            row.append(td.get_text().strip())
        i+=1
        # once the number of elements in the rows equals the number of columns, add it to the df and reset the row
        if i==num_cols:
            i=0
            df = df.append(pd.DataFrame(row).T)
            row=[]
    df.columns=columns
    return df
   

In [9]:
df = get_table_data(table)
cols = df.columns.values
cols[0]='Rank'
df.columns = cols
df.set_index('Rank', inplace=True) 
df

,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 population density,Location
Rank,,,,,,,,
1,New York[d],New York,"8,398,748","8,175,133",+2.74%,780.9 km2,"10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,"1,213.9 km2","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,588.7 km2,"4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,"1,651.1 km2","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,"1,340.6 km2","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...
...,...,...,...,...,...,...,...,...
310,Edison[ad],New Jersey,"100,693","99,967",+0.73%,78.0 km2,"1,309/km2",40°30′14″N 74°20′58″W﻿ / ﻿40.5040°N 74.3494°W﻿...
311,Woodbridge[ad],New Jersey,"100,450","99,585",+0.87%,60.3 km2,"1,680/km2",40°33′39″N 74°17′34″W﻿ / ﻿40.5607°N 74.2927°W﻿...
312,San Angelo,Texas,"100,215","93,200",+7.53%,155.1 km2,649/km2,31°26′28″N 100°27′02″W﻿ / ﻿31.4411°N 100.4505°...


Now we need to simplify this data to only keep what we need (City, State, 2018 estimated population, 2016 population density)

In [10]:
df_pop = df[ ['City','State[c]','2018estimate','2016 population density'] ]
# rename the columns
df_pop.columns = ['City','State','Population','Density']
df_pop

,City,State,Population,Density
Rank,,,,
1,New York[d],New York,"8,398,748","10,933/km2"
2,Los Angeles,California,"3,990,456","3,276/km2"
3,Chicago,Illinois,"2,705,994","4,600/km2"
4,Houston[3],Texas,"2,325,502","1,395/km2"
5,Phoenix,Arizona,"1,660,272","1,200/km2"
...,...,...,...,...
310,Edison[ad],New Jersey,"100,693","1,309/km2"
311,Woodbridge[ad],New Jersey,"100,450","1,680/km2"
312,San Angelo,Texas,"100,215",649/km2


Lastly, we will remove the /km2 from the density column and convert to type float

In [11]:
df_pop['Density']=df_pop['Density'].replace('/km2','',regex=True)
df_pop['City']=df_pop['City'].replace('\[.*\]','',regex=True)
df_pop

/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,State,Population,Density
Rank,,,,
1,New York,New York,"8,398,748","10,933"
2,Los Angeles,California,"3,990,456","3,276"
3,Chicago,Illinois,"2,705,994","4,600"
4,Houston,Texas,"2,325,502","1,395"
5,Phoenix,Arizona,"1,660,272","1,200"
...,...,...,...,...
310,Edison,New Jersey,"100,693","1,309"
311,Woodbridge,New Jersey,"100,450","1,680"
312,San Angelo,Texas,"100,215",649


#### Media Market Data

Next, we'll pull in the tv media market data from Wikipedia

In [12]:
tv_wiki = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_television_markets")
tv_soup = BeautifulSoup(tv_wiki.content,'html')
#tv_soup

In [13]:
# get the table reference
# By inspection of HTML, table we are interested in is stored in index 1
table = tv_soup.find_all('table')[1]
#table

In [14]:
def get_tv_table_data(table):
    
    # Extract the column names
    columns = []
    for c in table.find_all('th'):
        if 'scope' in c.attrs:
            pass
        else:
            columns.append(c.get_text().strip())
   
    # get the remaining elements based on the length of the column headers
    num_cols = len(columns)
    # create an empty dataframe with the columns discovered
    df = pd.DataFrame()
    
    i=0
    row=[]
    
    # get the market column based on the th tag
    markets = []
    for market in table.find_all('th'):
        if 'scope' in market.attrs:
            markets.append(market.get_text().strip())
   
    # get the remaining elements based on the length of the column headers
    num_cols = len(columns)
    # create an empty dataframe with the columns discovered
    df = pd.DataFrame()
    
    i=0
    j=0
    row=[]
       
    for td in table.find_all('td'):
        if i==1:
            try:
                row.append(markets[j])
                j+=1
                i+=1
            except:
                pass

        # if it's a name with a link, get the name out of the link otherwise just append it to the row
        if td.find('a')!=None:
            row.append(td.find('a').get_text())
        else:
            row.append(td.get_text().strip())
        i+=1
        # once the number of elements in the rows equals the number of columns, add it to the df and reset the row
        if i==num_cols:
            i=0
            df = df.append(pd.DataFrame(row).T)
            row=[]

    df.columns=columns
    df.set_index('Rank[1]', inplace=True)
    return df

In [15]:
df_tv = get_tv_table_data(table)
df_tv.head()

,Market,State,Counties (or county-equivalents) covered,TV households (2018–19),Local ABC affiliate,Local CBS affiliate,Local CW affiliate,Local Fox affiliate,Local NBC affiliate,Other significant stations[2]
Rank[1],,,,,,,,,,
1,New York,New York,Bronx,"7,100,300 (6.441%)",WABC-TV,WCBS-TV,WPIX,WNYW,WNBC,WFTY-DT
2,Los Angeles,California,Inyo,"5,276,600 (4.786%)",KABC-TV,KCBS-TV,KTLA,KTTV,KNBC,KCAL-TV
3,Chicago,Illinois,Cook,"3,251,370 (2.949%)",WLS-TV,WBBM-TV,WCIU-TV,WFLD,WMAQ-TV,WGN-TV
4,Philadelphia,Pennsylvania,Berks,"2,816,850 (2.555%)",WPVI-TV,KYW-TV,WPSG,WTXF-TV,WCAU,WFPA-CD
5,Dallas-Fort Worth,Texas,Anderson,"2,622,070 (2.378%)",WFAA,KTVT,KDAF,KDFW,KXAS-TV,KDFI


Now we'll remove the extraneous information and keep only what we need

In [16]:
df_tv_market = df_tv[ ['Market', 'State', 'TV households (2018–19)']]
df_tv_market

,Market,State,TV households (2018–19)
Rank[1],,,
1,New York,New York,"7,100,300 (6.441%)"
2,Los Angeles,California,"5,276,600 (4.786%)"
3,Chicago,Illinois,"3,251,370 (2.949%)"
4,Philadelphia,Pennsylvania,"2,816,850 (2.555%)"
5,Dallas-Fort Worth,Texas,"2,622,070 (2.378%)"
...,...,...,...
209,North Platte,Nebraska,"12,830 (0.012%)"
210,Glendive,Montana,"3,590 (0.003%)"
N/A,American Samoa,Tutuila,KVZK-4


#### Median Income Data

Next, we'll pull in the median income data

In [17]:
inc_wiki = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_metropolitan_areas_by_per_capita_income")
inc_soup = BeautifulSoup(inc_wiki.content,'html')
#inc_soup

In [18]:
# get the table reference
# By inspection of HTML, table we are interested in is stored in index 2
table = inc_soup.find_all('table')[2]
#table

In [19]:
df_inc = get_table_data(table)
df_inc.set_index('Rank', inplace=True)
df_inc.rename(columns={'Metropolitan statistical area':'Metro'}, inplace=True)
df_inc.drop('Population', axis=1,inplace=True)
df_inc

,Metro,Per capitaincome
Rank,,
1,"Washington-Arlington-Alexandria, D.C-Virginia-...","$47,411"
2,"San Jose-Santa Clara-Sunnyvale, California MSA","$40,392"
3,"Seattle-Tacoma-Bellevue, Washington MSA","$39,322"
4,"San Francisco-Oakland-Hayward, California MSA","$38,355"
5,"Boston–Worcester–Lawrence, Massachusetts–New H...","$37,311"
...,...,...
276,"McAllen–Edinburg–Mission, Texas MSA","$9,899"
277,"San Juan–Caguas–Arecibo, Puerto Rico CMSA","$9,140"
278,"Mayagüez, Puerto Rico MSA","$7,730"


### NBA Team Cities
Finally, we'll pull in the list of NBA team cities by CSV

In [20]:
df_teams = pd.read_csv('NBA_cities.csv')

# filter by current teams (where To = 2020)
df_teams = df_teams[ df_teams['To']==2020 ]
# get list of unique NBA cities
nba_cities = list(df_teams.Franchise.unique())
nba_cities

['Atlanta Hawks',
 'Boston Celtics',
 'Brooklyn Nets',
 'Charlotte Hornets',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Houston Rockets',
 'Indiana Pacers',
 'Los Angeles Clippers',
 'Los Angeles Lakers',
 'Memphis Grizzlies',
 'Miami Heat',
 'Milwaukee Bucks',
 'Minnesota Timberwolves',
 'New Orleans Pelicans',
 'New York Knicks',
 'Oklahoma City Thunder',
 'Orlando Magic',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Portland Trail Blazers',
 'Sacramento Kings',
 'San Antonio Spurs',
 'Toronto Raptors',
 'Utah Jazz',
 'Washington Wizards']

### Merge Data Sets for Use
Now we will merge all of the datasets into a usable dataframe.

We want to convert all numerical items to type float and add the per-capita income and TV market information to the exisintg population data

First we'll add the per capita income

In [21]:
# store metro and pci information
metros = list(df_inc.Metro)
pci = list(df_inc['Per capitaincome'])

# create a function to extract it
def get_merged_value(city,options,values):
    for i in range(len(options)):
        if city in options[i]:
            return values[i]
    return np.nan
            

df_pop['PCI'] = df_pop['City'].apply(lambda x: get_merged_value(x,metros,pci))
df_pop

/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,City,State,Population,Density,PCI
Rank,,,,,
1,New York,New York,"8,398,748","10,933","$24,581"
2,Los Angeles,California,"3,990,456","3,276","$21,170"
3,Chicago,Illinois,"2,705,994","4,600",NaN
4,Houston,Texas,"2,325,502","1,395","$21,701"
5,Phoenix,Arizona,"1,660,272","1,200","$21,907"
...,...,...,...,...,...
310,Edison,New Jersey,"100,693","1,309",NaN
311,Woodbridge,New Jersey,"100,450","1,680",NaN
312,San Angelo,Texas,"100,215",649,"$17,325"


In [22]:
# casting as type float
df_pop['Population'] = df_pop['Population'].str.replace(',','').astype('float')
df_pop['Density'] = df_pop['Density'].str.replace(',','').astype('float')
df_pop['PCI'] = df_pop['PCI'].str.replace(',','').str.replace('$','').astype('float')
# fixing the Nan, filling by state
df_data = df_pop.interpolate()
df_data.index = df_data.index.astype(int)
df_data.sort_index(inplace=True)
df_data

/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

,City,State,Population,Density,PCI
Rank,,,,,
1,New York,New York,8398748.0,10933.0,24581.00
2,Los Angeles,California,3990456.0,3276.0,21170.00
3,Chicago,Illinois,2705994.0,4600.0,21435.50
4,Houston,Texas,2325502.0,1395.0,21701.00
5,Phoenix,Arizona,1660272.0,1200.0,21907.00
...,...,...,...,...,...
310,Edison,New Jersey,100693.0,1309.0,18161.50
311,Woodbridge,New Jersey,100450.0,1680.0,17743.25
312,San Angelo,Texas,100215.0,649.0,17325.00


In [23]:
# Check the types that result
df_pop.dtypes

City           object
State          object
Population    float64
Density       float64
PCI           float64
dtype: object

Adding the tv market in a similar fashion

In [24]:
market = list(df_tv_market['Market'])
households = list(df_tv_market['TV households (2018–19)'])

df_data['TV'] = df_data['City'].apply(lambda x: get_merged_value(x,market,households))
df_data

,City,State,Population,Density,PCI,TV
Rank,,,,,,
1,New York,New York,8398748.0,10933.0,24581.00,"7,100,300 (6.441%)"
2,Los Angeles,California,3990456.0,3276.0,21170.00,"5,276,600 (4.786%)"
3,Chicago,Illinois,2705994.0,4600.0,21435.50,"3,251,370 (2.949%)"
4,Houston,Texas,2325502.0,1395.0,21701.00,"2,423,360 (2.198%)"
5,Phoenix,Arizona,1660272.0,1200.0,21907.00,"1,864,420 (1.691%)"
...,...,...,...,...,...,...
310,Edison,New Jersey,100693.0,1309.0,18161.50,NaN
311,Woodbridge,New Jersey,100450.0,1680.0,17743.25,NaN
312,San Angelo,Texas,100215.0,649.0,17325.00,"52,790 (0.048%)"


In [25]:
# fix the NaN by interpolation after sorting by population
df_data['TV'] = df_data['TV'].str.replace('\(.*\)','').str.replace(',','').astype('float')
# fixing the Nan, filling by state
df_data = df_data.interpolate()
df_data.sort_index(inplace=True)
df_data

,City,State,Population,Density,PCI,TV
Rank,,,,,,
1,New York,New York,8398748.0,10933.0,24581.00,7100300.0
2,Los Angeles,California,3990456.0,3276.0,21170.00,5276600.0
3,Chicago,Illinois,2705994.0,4600.0,21435.50,3251370.0
4,Houston,Texas,2325502.0,1395.0,21701.00,2423360.0
5,Phoenix,Arizona,1660272.0,1200.0,21907.00,1864420.0
...,...,...,...,...,...,...
310,Edison,New Jersey,100693.0,1309.0,18161.50,366170.0
311,Woodbridge,New Jersey,100450.0,1680.0,17743.25,209480.0
312,San Angelo,Texas,100215.0,649.0,17325.00,52790.0


Finally, let's determine if the city already has an NBA team

1 = YES

0 = NO

In [26]:
# define a function to add it
def nba_city(city,nba_cities):
    for n in nba_cities:
        if city in n:
            return 1
    return 0

df_data['NBA']=df_data['City'].apply(lambda x: nba_city(x,nba_cities))
df_data

,City,State,Population,Density,PCI,TV,NBA
Rank,,,,,,,
1,New York,New York,8398748.0,10933.0,24581.00,7100300.0,1
2,Los Angeles,California,3990456.0,3276.0,21170.00,5276600.0,1
3,Chicago,Illinois,2705994.0,4600.0,21435.50,3251370.0,1
4,Houston,Texas,2325502.0,1395.0,21701.00,2423360.0,1
5,Phoenix,Arizona,1660272.0,1200.0,21907.00,1864420.0,1
...,...,...,...,...,...,...,...
310,Edison,New Jersey,100693.0,1309.0,18161.50,366170.0,0
311,Woodbridge,New Jersey,100450.0,1680.0,17743.25,209480.0,0
312,San Angelo,Texas,100215.0,649.0,17325.00,52790.0,0


In [64]:
df_data.NBA.value_counts()

# set outlier manually (Golden State = San Francisco)
cols = df_data['City']=='San Francisco'
df_data.loc[cols,'NBA']=1
# set outlier manually (Utah = Salt Lake City)
cols = df_data['City']=='Salt Lake City'
df_data.loc[cols,'NBA']=1
# set outlier manually (Minnesota = Minneapolis)
cols = df_data['City']=='Minneapolis'
df_data.loc[cols,'NBA']=1
# set outlier manually (Indiana = Indianapolis)
cols = df_data['City']=='Indianapolis'
df_data.loc[cols,'NBA']=1

df_data

,City,State,Population,Density,PCI,TV,NBA
Rank,,,,,,,
1,New York,New York,8398748.0,10933.0,24581.00,7100300.0,1
2,Los Angeles,California,3990456.0,3276.0,21170.00,5276600.0,1
3,Chicago,Illinois,2705994.0,4600.0,21435.50,3251370.0,1
4,Houston,Texas,2325502.0,1395.0,21701.00,2423360.0,1
5,Phoenix,Arizona,1660272.0,1200.0,21907.00,1864420.0,1
...,...,...,...,...,...,...,...
310,Edison,New Jersey,100693.0,1309.0,18161.50,366170.0,0
311,Woodbridge,New Jersey,100450.0,1680.0,17743.25,209480.0,0
312,San Angelo,Texas,100215.0,649.0,17325.00,52790.0,0


Since most of the smaller cities can't actually support an NBA team, we will restrict the list to the 120 largest cities. This will also help us reduce the size of the data set when we start adding the location data from FourSquare

In [65]:
# take only the top 120 largest cities to both reduce data size and the realization that small cities 
# won't be able support an NBA franchse
city_data = df_data.iloc[0:120]
city_data

,City,State,Population,Density,PCI,TV,NBA
Rank,,,,,,,
1,New York,New York,8398748.0,10933.0,24581.0,7.100300e+06,1
2,Los Angeles,California,3990456.0,3276.0,21170.0,5.276600e+06,1
3,Chicago,Illinois,2705994.0,4600.0,21435.5,3.251370e+06,1
4,Houston,Texas,2325502.0,1395.0,21701.0,2.423360e+06,1
5,Phoenix,Arizona,1660272.0,1200.0,21907.0,1.864420e+06,1
...,...,...,...,...,...,...,...
116,Amarillo,Texas,199924.0,760.0,18247.0,1.758800e+05,0
117,Yonkers,New York,199663.0,4307.0,25323.0,1.936967e+05,0
118,Aurora,Illinois,199602.0,1729.0,32399.0,2.115133e+05,0


### Get the City Longitude and Latitude

In [66]:
#!{sys.executable} -m pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

In [67]:
# create a function to add the latitude and longitude for each city
def get_long_lat(city,state,long=1):
    address = str(city)+', '+str(state)
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    if long==1:
        return longitude
    else:
        return latitude
   
# get_long_lat('San Jose','California',0)

In [68]:
# add the long and lat to the dataframe
city_data['Long'] = city_data.apply(lambda x: get_long_lat(x['City'],x['State'],1),axis=1)
city_data['Lat'] = city_data.apply(lambda x: get_long_lat(x['City'],x['State'],0),axis=1)
city_data

/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/daniel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,State,Population,Density,PCI,TV,NBA,Long,Lat
Rank,,,,,,,,,
1,New York,New York,8398748.0,10933.0,24581.0,7.100300e+06,1,-74.006015,40.712728
2,Los Angeles,California,3990456.0,3276.0,21170.0,5.276600e+06,1,-118.242767,34.053691
3,Chicago,Illinois,2705994.0,4600.0,21435.5,3.251370e+06,1,-87.624421,41.875562
4,Houston,Texas,2325502.0,1395.0,21701.0,2.423360e+06,1,-95.367697,29.758938
5,Phoenix,Arizona,1660272.0,1200.0,21907.0,1.864420e+06,1,-112.077346,33.448587
...,...,...,...,...,...,...,...,...,...
116,Amarillo,Texas,199924.0,760.0,18247.0,1.758800e+05,0,-101.833825,35.207219
117,Yonkers,New York,199663.0,4307.0,25323.0,1.936967e+05,0,-73.898747,40.931210
118,Aurora,Illinois,199602.0,1729.0,32399.0,2.115133e+05,0,-88.314754,41.757170


Calculate the distance to closest NBA city

In [73]:
from geopy import distance

In [94]:
# define a function for calculating the min distance between a non-NBA city and all other NBA cities
def min_distance(lat,long,nba_cities):
    min_d = 999999
    city_coord = (lat,long)
    for i in range(len(nba_cities)):
        c_coord = (nba_cities.iloc[i,7],nba_cities.iloc[i,6])
        d = distance.distance(city_coord,c_coord).km
        if d<min_d:
            min_d = d
    return min_d

In [95]:
# collect a dataframe of only NBA cities
nba_cities = city_data[city_data['NBA']==1]

city_data['MinDistance'] = city_data.apply(lambda x: min_distance(x['Lat'],x['Long'],nba_cities),axis=1)
city_data

/home/daniel/.local/lib/python3.6/site-packages/geopy/point.py:405: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


ValueError: ('Latitude must be in the [-90; 90] range.', 'occurred at index 1')

Let's now create a map of the US with the city data plotted 

In [69]:
# create map of US
latitude = 38.8777
longitude = -97.38
map_us = folium.Map(location=[latitude, longitude], zoom_start=4)

# add city markers to map, color them Red if they are already an NBA city and Blue otherwise
for lat, lng, label, nba in zip(city_data['Lat'], city_data['Long'], city_data['City'], city_data['NBA']):
    c = 'blue'
    r = 3
    if nba==1:
        c='red'
        r=5
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=r,
        popup=label,
        color=c,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        legend_name='NBA Team',
        parse_html=False).add_to(map_us)  

# add a legend

map_us


### Adding the FourSquare Data

In [70]:
CLIENT_ID = 'X1RDABIJGUOLEDZZHTFOOIME4KPAMKJPOXRRZZLYWJD2NBHT' # your Foursquare ID
CLIENT_SECRET = 'TNMIFQJTTJ3OIHUHPRA4XPZOXLAXKC40BYPXUH3PTBXFWTEU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X1RDABIJGUOLEDZZHTFOOIME4KPAMKJPOXRRZZLYWJD2NBHT
CLIENT_SECRET:TNMIFQJTTJ3OIHUHPRA4XPZOXLAXKC40BYPXUH3PTBXFWTEU


Let's write a function to get nearby venues for each city in our list.
We will consider venues within 5 km of the city center, limiting our response to 1000

In [72]:
radius=5000
LIMIT=100
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&intent=browse&radius={}&limit={}'.format(
     CLIENT_ID, 
     CLIENT_SECRET, 
     VERSION, 
     latitude, 
     longitude, 
     radius, 
     LIMIT)
            
# make the GET request
results = requests.get(url).json() 
results


{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5d7bec416e4650002cd85589'},
 'response': {}}

In [34]:
def getCityVenues(cities, states, latitudes, longitudes, radius=1000, LIMIT=100):
    
    venues_list=[]
    for city, state, lat, lng in zip(cities, states, latitudes, longitudes):
        print(city, state)
        
        # will repeat this request 5 times per city to get enough location data
        # once at the base lat/long and the one at 0.1 increments at each
        lats = [lat,lat,lat,lat+0.2,lat-0.2]
        lngs = [lng,lng+0.2,lng-0.2,lng,lng]
        for la, ln in zip(lats,lngs):
            print(la,ln)
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                la, 
                ln, 
                radius, 
                LIMIT)
            
            try:
                # make the GET request
                results = requests.get(url).json()["response"]['groups'][0]['items']
        
                # return only relevant information for each nearby venue
                venues_list.append([(
                    city,
                    state, 
                    lat, 
                    lng, 
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name']) for v in results])
            except Exception as e:
                print(e)
                
    city_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    print(city_venues)
    city_venues.columns = ['City', 
                           'State',
                           'Latitude', 
                           'Longitude', 
                           'Venue', 
                           'Venue Lat', 
                           'Venue Long', 
                           'Venue Category']
    
    return(city_venues)

In [35]:
#small = city_data.iloc[0:1]
# now let's collect the city venues
city_venues = getCityVenues(cities=city_data['City'], states=city_data['State'],
                                   latitudes=city_data['Lat'],
                                   longitudes=city_data['Long']
                                  )

#city_venues = getCityVenues(cities=small['City'], states=small['State'],
#                                   latitudes=small['Lat'],
#                                   longitudes=small['Long']
#                                  )

city_venues

New York New York
40.7127281 -74.0060152
40.7127281 -73.80601519999999
'groups'
40.7127281 -74.2060152
'groups'
40.9127281 -74.0060152
'groups'
40.5127281 -74.0060152
'groups'
Los Angeles California
34.0536909 -118.2427666
'groups'
34.0536909 -118.0427666
'groups'
34.0536909 -118.4427666
'groups'
34.2536909 -118.2427666
'groups'
33.8536909 -118.2427666
'groups'
Chicago Illinois
41.8755616 -87.6244212
'groups'
41.8755616 -87.4244212
'groups'
41.8755616 -87.8244212
'groups'
42.0755616 -87.6244212
'groups'
41.675561599999995 -87.6244212
'groups'
Houston Texas
29.7589382 -95.3676974
'groups'
29.7589382 -95.1676974
'groups'
29.7589382 -95.5676974
'groups'
29.9589382 -95.3676974
'groups'
29.5589382 -95.3676974
'groups'
Phoenix Arizona
33.4485866 -112.0773456
'groups'
33.4485866 -111.8773456
'groups'
33.4485866 -112.2773456
'groups'
33.6485866 -112.0773456
'groups'
33.248586599999996 -112.0773456
'groups'
Philadelphia Pennsylvania
39.9527237 -75.1635262
'groups'
39.9527237 -74.9635262
'groups

'groups'
36.8529841 -76.1774183
'groups'
37.0529841 -75.9774183
'groups'
36.6529841 -75.9774183
'groups'
Oakland California
37.8044557 -122.2713563
'groups'
37.8044557 -122.07135629999999
'groups'
37.8044557 -122.4713563
'groups'
38.0044557 -122.2713563
'groups'
37.604455699999995 -122.2713563
'groups'
Minneapolis Minnesota
44.9706705 -93.268217735811
'groups'
44.9706705 -93.068217735811
'groups'
44.9706705 -93.46821773581101
'groups'
45.1706705 -93.268217735811
'groups'
44.770670499999994 -93.268217735811
'groups'
Tulsa Oklahoma
36.1556805 -95.9929113
'groups'
36.1556805 -95.7929113
'groups'
36.1556805 -96.1929113
'groups'
36.355680500000005 -95.9929113
'groups'
35.9556805 -95.9929113
'groups'
Arlington Texas
32.701939 -97.105624826688
'groups'
32.701939 -96.905624826688
'groups'
32.701939 -97.305624826688
'groups'
32.901939000000006 -97.105624826688
'groups'
32.501939 -97.105624826688
'groups'
Tampa Florida
27.9477595 -82.458444
'groups'
27.9477595 -82.258444
'groups'
27.9477595 -82.

'groups'
33.3527632 -111.5890373
'groups'
33.3527632 -111.9890373
'groups'
33.5527632 -111.7890373
'groups'
33.152763199999995 -111.7890373
'groups'
Winston–Salem North Carolina
36.0998131 -80.2440518
'groups'
36.0998131 -80.04405179999999
'groups'
36.0998131 -80.4440518
'groups'
36.2998131 -80.2440518
'groups'
35.899813099999996 -80.2440518
'groups'
North Las Vegas Nevada
36.2008371 -115.1120958
'groups'
36.2008371 -114.9120958
'groups'
36.2008371 -115.31209580000001
'groups'
36.400837100000004 -115.1120958
'groups'
36.0008371 -115.1120958
'groups'
Norfolk Virginia
36.8462923 -76.2929252
'groups'
36.8462923 -76.0929252
'groups'
36.8462923 -76.4929252
'groups'
37.046292300000005 -76.2929252
'groups'
36.6462923 -76.2929252
'groups'
Chesapeake Virginia
36.7183708 -76.2466798
'groups'
36.7183708 -76.04667979999999
'groups'
36.7183708 -76.4466798
'groups'
36.918370800000005 -76.2466798
'groups'
36.5183708 -76.2466798
'groups'
Garland Texas
32.912624 -96.6388833
'groups'
32.912624 -96.43888

,City,State,Latitude,Longitude,Venue,Venue Lat,Venue Long,Venue Category
0,New York,New York,40.712728,-74.006015,The Bar Room at Temple Court,40.711448,-74.006802,Hotel Bar
1,New York,New York,40.712728,-74.006015,The Beekman - A Thompson Hotel,40.711173,-74.006702,Hotel
2,New York,New York,40.712728,-74.006015,Alba Dry Cleaner & Tailor,40.711434,-74.006272,Laundry Service
3,New York,New York,40.712728,-74.006015,The Wooly Daily,40.712137,-74.008395,Coffee Shop
4,New York,New York,40.712728,-74.006015,City Hall Park,40.712415,-74.006724,Park
...,...,...,...,...,...,...,...,...
95,New York,New York,40.712728,-74.006015,Oliver Coffee,40.712986,-73.998106,Coffee Shop
96,New York,New York,40.712728,-74.006015,Sam's Falafel,40.708744,-74.011458,Food Truck
97,New York,New York,40.712728,-74.006015,Apothéke,40.714470,-73.998018,Cocktail Bar
98,New York,New York,40.712728,-74.006015,Crown Shy,40.706187,-74.007490,Restaurant


In [36]:
city_venues

,City,State,Latitude,Longitude,Venue,Venue Lat,Venue Long,Venue Category
0,New York,New York,40.712728,-74.006015,The Bar Room at Temple Court,40.711448,-74.006802,Hotel Bar
1,New York,New York,40.712728,-74.006015,The Beekman - A Thompson Hotel,40.711173,-74.006702,Hotel
2,New York,New York,40.712728,-74.006015,Alba Dry Cleaner & Tailor,40.711434,-74.006272,Laundry Service
3,New York,New York,40.712728,-74.006015,The Wooly Daily,40.712137,-74.008395,Coffee Shop
4,New York,New York,40.712728,-74.006015,City Hall Park,40.712415,-74.006724,Park
...,...,...,...,...,...,...,...,...
95,New York,New York,40.712728,-74.006015,Oliver Coffee,40.712986,-73.998106,Coffee Shop
96,New York,New York,40.712728,-74.006015,Sam's Falafel,40.708744,-74.011458,Food Truck
97,New York,New York,40.712728,-74.006015,Apothéke,40.714470,-73.998018,Cocktail Bar
98,New York,New York,40.712728,-74.006015,Crown Shy,40.706187,-74.007490,Restaurant


Next, we need to extract and one-hot the locale information

In [37]:
# define a function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
city_venues['Venue Category'].value_counts()

KeyboardInterrupt: 

In [ ]:
# one hot encoding
cities_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add city and state information back in
cities_onehot['City'] = city_venues['City'] 
cities_onehot['State'] = city_venues['State'] 
cities_onehot.head()

In [ ]:
# move city and state back to the front
fixed_columns = [cities_onehot.columns[-2]] + [cities_onehot.columns[-1]] + list(cities_onehot.columns[:-2])
cities_onehot = cities_onehot[fixed_columns]

cities_onehot.head()

Grouping the data by city

In [ ]:
grouped_city_venues = cities_onehot.groupby('City').mean().reset_index()
grouped_city_venues.head()

In [ ]:
grouped_city_venues.shape

Finish merging the data sets

In [ ]:
# merge with the city data 
df_cities = city_data.merge(grouped_city_venues,on='City',how='left')

### Methodology -- Exploratory Data Analyis

Let's take an initial exploratory look at the data. Information and visualization that will help us see what NBA cities look like include
- Gropued statistics population, per capita income and TV market size for an NBA city
- Box plots of population, income and TV market size,colored by if it is an NBA city or not
- Scatter plot of TV market size vs population size colored by NBA city

Looking at simple statistics first:

In [ ]:
# setting up the formatting
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
df_cities.groupby('NBA')[['Population','PCI','TV']].describe()

The main takeaways we can pull from this summary are
- NBA cities tend to have higher populations but not all high population cities have an NBA city
- statistical difference in per capita income are negligible for the two sets
- The TV market size is much larger for an NBA dity than it is for a non-NBA city

Let's plot the data to get a more clear picture of these trends

In [ ]:
# pull in the required modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#!{sys.executable} -m pip install seaborn
import seaborn as sns

Plot the box-plots of interest

In [ ]:
# plotting the population box plots
ax = sns.boxplot(x='NBA',y='Population', data=df_cities)
ax.set_xlabel('NBA City (0=NO, 1=YES)')
ax.set_ylabel('Population')

In [ ]:
ax = sns.boxplot(x='NBA',y='TV', data=df_cities)
ax.set_ylabel('TV Market Size')
ax.set_xlabel('NBA City (0=NO, 1=YES)')

In [ ]:
ax = sns.boxplot(x='NBA',y='PCI', data=df_cities)
ax.set_ylabel('Per Capita Income [$]')
ax.set_xlabel('NBA City (0=NO, 1=YES)')

In [ ]:
# Now let's plot the TV Market vs city data, colored by NBA city
plt.figure()
sc = plt.scatter(df_cities['Population'],df_cities['TV'], c=df_cities['NBA'])
plt.ylabel('TV Market Size')
plt.xlabel('Population')
plt.legend(loc='best')
plt.grid(True)
plt.colorbar(sc)
plt.show()

In [ ]:
# lastly, lets depict the top 10 venues by city
